# Proyecto - Clase 3

Para este proyecto, vamos a trabajar con datos del Banco Mundial, referida a estadísticas de educación a nivel internacional. Pueden leer más de la misma en https://www.kaggle.com/theworldbank/world-bank-intl-education.

Ya está preprocesada por nosotros, y está incluida en el repositorio del curso. El nombre del archivo es `EdStats.csv`, pero también hay una versión reducida que tiene sólo 1000 líneas del archivo original y se llama `EdStats_prueba.csv`. Usaremos este último en la etapa de desarrollo del código. Ambos archivos están en el subdirectorio `datos` del directorio `clase3`.

El objetivo de este proyecto es familiarizarnos con la creación de clases y objetos, y la filosofía detrás de su uso. Para ello vamos a implementar algunas clases de Python para que nos ayuden a estructurar estos datos (Ejercicios 1-4). El resto de los ejercicios les quedan de tarea.

## Descripción de los datos

Vamos a usar un archivo csv con las siguientes columnas:
- **Country Name**: tiene información básica de los países.
- **Country Code**: tiene, por cada fila, una estadística para un dado país.
- **Indicator Code**: código que identifica al indicador que se provee en esa fila.
- **value_2014**: valor del indicador anterior para el año 2014

### Primero definamos algunas constantes (los paths de los archivos)

In [1]:
ARCHIVO_DATOS = "datos/EdStats.csv"
ARCHIVO_DATOS_PRUEBA = "datos/EdStats_prueba.csv"

### Ejercicio 1: 

Hagamos una clase muy básica que represente a un país y llamémosla `Pais`. Vamos a crear un constructor, en el que por ahora incluiremos sólo nombre y abreviatura del país, y un diccionario vacío donde más adelante vamos a depositar las estadísticas que tiene la tabla 2.

In [2]:
class Pais:
    
    def __init__(self, nombre, abreviatura):
        self.nombre = nombre
        self.abreviatura = abreviatura
        self.estadisticas = {}
        
    # agregué este método para que podamos imprimir el país de una manera amigable
    def __str__(self):
        return self.nombre

### Ejercicio 2:

Ahora creemos una clase que represente al mundo. Llamémosla `Mundo`. No va a ser más que un diccionario, llamado `paises`, cuyas claves serán los nombres de los países y los valores serán cada uno un objeto de clase `Pais`. Luego agregaremos métodos para interactuar con ella (con la clase `Mundo`).


Dado que el mundo es uno solo, *no necesitamos definir un constructor* (es decir, no vamos a instanciar distintos objetos de clase `Mundo`).

En principio este diccionario va a estar vacío, y vamos a agregar países a medida que nos los vayamos encontrando en la tabla. Para hacerlo, vamos a crear dos métodos:

- Uno se va a encargar de agregar un país al diccionario. Llamémoslo `agregar_pais`.
- Otro se va a encargar de agregar una estadística el país correspondiente. Sin embargo, si ese país no se encuentra ya en el diccionario, previamente tiene que llamar al método anterior para agregarlo. Para determinar este comportamiento usaremos una instrucción try/except. A este método lo llamaremos `agregar_estadistica`.

In [3]:
class Mundo:
    
    paises = {}
    
    def agregar_pais(nombre_del_pais, abr_del_pais):
        Mundo.paises[abr_del_pais] = Pais(nombre_del_pais, abr_del_pais)
    
    def agregar_estadistica(nombre_del_pais, abr_del_pais, codigo_de_indicador, valor):
        
        try:
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor
        except KeyError:
            Mundo.agregar_pais(nombre_del_pais, abr_del_pais)
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor

### Ejercicio 3:
Ahora creemos una clase para ayudarnos a leer el archivo `csv`. Llamémosla `LectorDeTabla`. Noten que esto no es más que una especie de sustituto del `csv.reader` que ya hemos visto y usado.

La idea es que podamos utilizar esta clase para instanciar objetos que representen un dado archivo `csv` y, mediante métodos, simplificar el proceso de extraer los campos que nos interesan.

Por lo tanto, vamos a implementar varios métodos:
- El **constructor**: va a recibir el nombre del archivo como argumento y, opcionalmente, el carácter que se usa como separador de celdas.
- Un método llamado `leer_siguiente_linea`, no va a recibir ningún argumento (además del objeto en sí), y va a devolver una lista, cuyos elementos son los valores de cada una de las celdas de esa línea.
- `extraer_pais` va a devolver el país correspondiente a la línea actual.
- `extraer_estadistica` va a devolver la estadística correspondiente a la línea actual.

Para ayudarnos con esto, vamos a utilizar algunos métodos de procesamiento de strings:
- `str.split`: recibe un string "separador" (por ejemplo, `,`) y devuelve una lista producida particionando la string de acuerdo a ese separador.
- `str.strip`: devuelve un string pero sin los espacios en blanco o saltos de línea que pueda haber en los extremos.

#### Antes de escribir la clase, hagamos unas pruebas:

Veamos cómo funcionan `split` y `strip`

In [4]:
"Hola que tal".split(" ")

In [5]:
"Hola, que tal".split(",")

In [6]:
"    Hola que tal   ".strip()

Ahora veamos cómo leer un archivo línea por línea

In [7]:
f = open(ARCHIVO_DATOS_PRUEBA, "r")

In [8]:
f.readline() # Leemos el encabezado (pero no lo guardamos porque no nos interesa)

In [9]:
line = f.readline()
line # hay un '\n' (un salto de línea) al final, por eso necesitamos el strip

'Arab World,ARB,UIS.NERA.2,\n'

Con esta instrucción obtenemos una lista con los componentes de la línea en cuestión

In [10]:
componentes = line.strip().split(",")
componentes

['Arab World', 'ARB', 'UIS.NERA.2', '']

In [11]:
class LectorDeTabla:
    
    def __init__(self, archivo, separador=","):
        self.archivo = open(archivo, "r")
        self.separador = separador
        self.linea_actual = None
        
    def leer_siguiente_linea(self):
        self.linea_actual = self.archivo.readline().strip().split(self.separador)
    
    def extraer_pais(self):
        return Pais(self.linea_actual[0], self.linea_actual[1])
    
    def extraer_estadistica(self):
        # En la clase había hecho que este método devuelva un diccionario
        # Ahora lo cambié por una tupla por simplicidad
        codigo_de_indicador, valor = self.linea_actual[2], self.linea_actual[3]
        return (codigo_de_indicador, valor)
    

In [12]:
tabla = LectorDeTabla(ARCHIVO_DATOS)
tabla.leer_siguiente_linea() # Leemos la primera línea, que es el encabezado

In [13]:
tabla.leer_siguiente_linea()

In [14]:
pais = tabla.extraer_pais()
print(pais)

Arab World


### Ejercicio 4:
Ahora "juntemos" las dos clases: vamos a utilizar `LectorDeTabla` para ir leyendo el archivo línea por línea y extraer los campos que nos interesan. 

Para hacer esto, vamos a ampliar la clase `Mundo`, creando un método llamado `cargar_contenido` que recibe el nombre de nuestro archivo de datos y nos vuelca todo su contenido en la clase `Mundo`.
Es decir, queremos un método que nos permita cargar todo el contenido del archivo ejecutando simplemente
`Mundo.cargar_contenido(<ARCHIVO>)`

In [15]:
class Mundo:
    
    paises = {}
    
    def agregar_pais(nombre_del_pais, abr_del_pais):
        Mundo.paises[abr_del_pais] = Pais(nombre_del_pais, abr_del_pais)
    
    def agregar_estadistica(nombre_del_pais, abr_del_pais, codigo_de_indicador, valor):
        
        try:
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor
        except KeyError:
            # Si el país no está en el diccionario paises, se agrega...
            Mundo.agregar_pais(nombre_del_pais, abr_del_pais)
            # ... y luego se agrega la estadística correpondiente
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor
            
    def cargar_contenido(archivo):
        
        lector = LectorDeTabla(archivo)
        lector.leer_siguiente_linea() # leemos el encabezado
        
        while True:            
            
            lector.leer_siguiente_linea()
            
            # Cuando llegamos al final del archivo, lector.linea_actual vale [''],
            # entonces podemos chequearlo con este if
            if len(lector.linea_actual) == 1: 
                return
            
            pais = lector.extraer_pais()        
            estadistica = lector.extraer_estadistica()
            Mundo.agregar_estadistica(pais.nombre, pais.abreviatura, estadistica[0], estadistica[1])    

Acá usamos el archivo completo.

In [18]:
Mundo.cargar_contenido(ARCHIVO_DATOS)

dict_keys(['ARB', 'EAS', 'EAP', 'EMU', 'ECS', 'ECA', 'EUU', 'HPC', 'HIC', 'LCN', 'LAC', 'LDC', 'LMY', 'LIC', 'LMC', 'MEA', 'MNA', 'MIC', 'NAC', 'OED', 'SAS', 'SSF', 'SSA', 'UMC', 'WLD', 'AFG', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'VGB', 'BRN', 'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CYM', 'CAF', 'TCD', 'CHI', 'CHL', 'CHN', 'COL', 'COM', 'CRI', 'CIV', 'HRV', 'CUB', 'CUW', 'CYP', 'CZE', 'DNK', 'DJI', 'DMA', 'DOM', 'ECU', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'FRO', 'FJI', 'FIN', 'FRA', 'PYF', 'GAB', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HUN', 'ISL', 'IND', 'IDN', 'IRQ', 'IRL', 'IMN', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'XKX', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LIE', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL', 'MRT', 'MUS', 'M

### Ejercicio 5:
Ampliar la clase `Mundo` agregando un método llamado `lista_de_paises` que devuelva una **lista ordenada alfabéticamente** de todos los países que contiene dicha clase. La lista debe contener los **nombres completos** de los países (no las abreviaturas). Ayuda: usar la función `sorted`. Este método se puede escribir en una línea de código.

### Ejercicio 6:
Ampliar la clase Mundo agregando un método llamado `lista_de_estadisticas` que devuelva los nombres de todas las estadísticas que tiene un dado país (que se le debe pasar como argumento a dicho método). Si se provee un país que no pertenece al diccionario, debe imprimir un mensaje que diga `El país <PAIS> no existe en el Mundo` (pero **no** debe producir error). Ayuda: usar `try/except`.

### Ejercicio 7:
Ampliar la clase `Mundo` agregando un método llamado `gasto_educ_total`, que reciba como argumento un país y devuelva el gasto total en educación del país en dólares, usando para calcularlo 1) el PBI en dólares y 2) el gasto en educación como porcentaje del PBI. 
Si el país no tiene los datos correspondientes necesarios para calcular esto, nuevamente no debe producir error, sino imprimir un mensaje que diga `El país <PAIS> no tiene datos suficientes para calcular el gasto total en educación.`

Nombres de los códigos que tienen que utilizar:

- `NY.GDP.MKTP.CD`: PBI en dólares actuales.

- `SE.XPD.TOTL.GD.ZS`: Gasto total en educación como porcentaje del PBI.

### Resuelvan los 3 ejercicios dentro de esta celda:

In [17]:
class Mundo:
    
    paises = {}
    
    def agregar_pais(nombre_del_pais, abr_del_pais):
        Mundo.paises[abr_del_pais] = Pais(nombre_del_pais, abr_del_pais)
    
    def agregar_estadistica(nombre_del_pais, abr_del_pais, codigo_de_indicador, valor):
        
        try:
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor
        except KeyError:
            # Si el país no está en el diccionario paises, se agrega...
            Mundo.agregar_pais(nombre_del_pais, abr_del_pais)
            # ... y luego se agrega la estadística correpondiente
            Mundo.paises[abr_del_pais].estadisticas[codigo_de_indicador] = valor
            
    def cargar_contenido(archivo):
        
        lector = LectorDeTabla(archivo)
        lector.leer_siguiente_linea() # leemos el encabezado
        
        while True:            
            
            lector.leer_siguiente_linea()
            
            # Cuando llegamos al final del archivo, lector.linea_actual vale [''],
            # entonces podemos chequearlo con este if
            if len(lector.linea_actual) == 1: 
                return
            
            pais = lector.extraer_pais()        
            estadistica = lector.extraer_estadistica()
            Mundo.agregar_estadistica(pais.nombre, pais.abreviatura, estadistica[0], estadistica[1])    

            
    # Ejercicio 5
    def lista_de_paises():
        # SU CÓDIGO ACÁ
        pass # esto lo tienen que eliminar al final, es sólo para que no les dé error al correr esta celda como está

    
    # Ejercicio 6
    def lista_de_estadisticas(abr_del_pais):
        # SU CÓDIGO ACÁ
        
        #try:
        #
        #except:
        #
        
        pass # ídem arriba 

    
    # Ejercicio 7
    def gasto_educ_total(abr_del_pais):
        # SU CÓDIGO ACÁ
        
        #try:
        #
        #except:
        #
        
        pass # ídem arriba